In [2]:
### Exercice fait main pour les GOTY ###

## 1) Packages et instanciation du client Spark Session
from pyspark.sql import SparkSession
import configparser
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.mllib.linalg import Vectors


import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import format_number
from pyspark.sql.functions import max, min
from pyspark.sql.functions import mean
from pyspark.sql.functions import corr
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.types import IntegerType, DateType, DoubleType

import folium
from folium.plugins import MarkerCluster
import pandas as pd

spark = SparkSession.builder.appName('Brisbane-city-bike').getOrCreate()
sc = spark.sparkContext

In [80]:
## 2) Créer un fichier properties.conf contenant les informations relatives à vos paramètres du programme en dur

import configparser
config = configparser.ConfigParser()
config.read('properties.conf')

path_to_input_data= config['Bristol-city-bike']['Input-data']
path_to_output_data= config['Bristol-city-bike']['Output-data']
num_partition_kmeans = config['Bristol-city-bike']['Kmeans-level']


In [81]:
## 3) Importer le json avec spark
Brisbane = spark.read.json(path_to_input_data)
Brisbane.show(3)

+--------------------+----------+----------+--------------------+------+
|             address|  latitude| longitude|                name|number|
+--------------------+----------+----------+--------------------+------+
|Lower River Tce /...|-27.482279|153.028723|122 - LOWER RIVER...|   122|
|Main St / Darragh St| -27.47059|153.036046|91 - MAIN ST / DA...|    91|
|Sydney St Ferry T...|-27.474531|153.042728|88 - SYDNEY ST FE...|    88|
+--------------------+----------+----------+--------------------+------+
only showing top 3 rows



In [82]:
## 4) Créer un nouveau data frame Kmeans-df contenant seulement les variables latitude et longitude. 

Kmeans_df = Brisbane["latitude", "longitude"]

Kmeans_df.show(10)

+----------+----------+
|  latitude| longitude|
+----------+----------+
|-27.482279|153.028723|
| -27.47059|153.036046|
|-27.474531|153.042728|
|-27.461881|153.046986|
|-27.469658|153.016696|
| -27.48172| 153.00436|
|-27.493626|153.001482|
|-27.476076|153.002459|
|-27.493963|153.011938|
|-27.482197|153.020894|
+----------+----------+
only showing top 10 rows



In [87]:
## 5) K-means

features = ("latitude", "longitude")
vecteurs = VectorAssembler(inputCols = features, outputCol = "features")
Brisbane_df = vecteurs.transform(Kmeans_df)
Brisbane_df.show()


kmeans = KMeans().setK(int(num_partition_kmeans)).setSeed(1)  # 3 clusters
model = kmeans.fit(Brisbane_df.select('features'))
fitted = model.transform(Brisbane_df)

+----------+----------+--------------------+
|  latitude| longitude|            features|
+----------+----------+--------------------+
|-27.482279|153.028723|[-27.482279,153.0...|
| -27.47059|153.036046|[-27.47059,153.03...|
|-27.474531|153.042728|[-27.474531,153.0...|
|-27.461881|153.046986|[-27.461881,153.0...|
|-27.469658|153.016696|[-27.469658,153.0...|
| -27.48172| 153.00436|[-27.48172,153.00...|
|-27.493626|153.001482|[-27.493626,153.0...|
|-27.476076|153.002459|[-27.476076,153.0...|
|-27.493963|153.011938|[-27.493963,153.0...|
|-27.482197|153.020894|[-27.482197,153.0...|
|-27.465226|153.050864|[-27.465226,153.0...|
|-27.468447|153.024662|[-27.468447,153.0...|
|-27.473021|153.025988|[-27.473021,153.0...|
|-27.457825|153.036866|[-27.457825,153.0...|
| -27.48148| 153.02368|[-27.48148,153.02...|
|-27.467464|153.022094|[-27.467464,153.0...|
|-27.499963|153.017633|[-27.499963,153.0...|
|-27.490776|152.994747|[-27.490776,152.9...|
|-27.458199|153.041688|[-27.458199,153.0...|
|-27.48180

In [88]:
## 6) Quels sont les noms des colonnes de fitted ? vérifier qu’il s’agit de longitude, latitude, features, predictions.

fitted.show(30)

+----------+----------+--------------------+----------+
|  latitude| longitude|            features|prediction|
+----------+----------+--------------------+----------+
|-27.482279|153.028723|[-27.482279,153.0...|         2|
| -27.47059|153.036046|[-27.47059,153.03...|         2|
|-27.474531|153.042728|[-27.474531,153.0...|         1|
|-27.461881|153.046986|[-27.461881,153.0...|         1|
|-27.469658|153.016696|[-27.469658,153.0...|         2|
| -27.48172| 153.00436|[-27.48172,153.00...|         0|
|-27.493626|153.001482|[-27.493626,153.0...|         0|
|-27.476076|153.002459|[-27.476076,153.0...|         0|
|-27.493963|153.011938|[-27.493963,153.0...|         0|
|-27.482197|153.020894|[-27.482197,153.0...|         2|
|-27.465226|153.050864|[-27.465226,153.0...|         1|
|-27.468447|153.024662|[-27.468447,153.0...|         2|
|-27.473021|153.025988|[-27.473021,153.0...|         2|
|-27.457825|153.036866|[-27.457825,153.0...|         1|
| -27.48148| 153.02368|[-27.48148,153.02...|    

In [89]:
## 7) Déterminer les longitudes et latitudes moyennes pour chaque groupe en utilisant spark DSL et SQL. comparer les résultats

fitted.groupBy("prediction").avg("latitude","longitude").show()

fitted.createOrReplaceTempView('Table')
spark.sql("""SELECT MEAN(latitude) as Latitude, MEAN(longitude) as Longitude, prediction FROM Table GROUP BY prediction""").show()

+----------+-------------------+------------------+
|prediction|      avg(latitude)|    avg(longitude)|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+

+-------------------+------------------+----------+
|           Latitude|         Longitude|prediction|
+-------------------+------------------+----------+
|-27.460240636363633|153.04186302272726|         1|
| -27.47255990624999|   153.02594553125|         2|
|-27.481218536585374|153.00572882926832|         0|
+-------------------+------------------+----------+



In [94]:
## 8) Bonus:  Faire une visualisation dans une map avec le package leaflet

pandasDF = fitted.toPandas() # Transformation en dataframe Pandas 
pandasDF

,latitude,longitude,features,prediction
0,-27.482279,153.028723,"[-27.482279, 153.028723]",2
1,-27.470590,153.036046,"[-27.47059, 153.036046]",2
2,-27.474531,153.042728,"[-27.474531, 153.042728]",1
3,-27.461881,153.046986,"[-27.461881, 153.046986]",1
4,-27.469658,153.016696,"[-27.469658, 153.016696]",2
...,...,...,...,...
144,-27.465880,153.026861,"[-27.46588, 153.026861]",2
145,-27.470110,153.025627,"[-27.47011, 153.025627]",2
146,-27.462665,153.029087,"[-27.462665, 153.029087]",2
147,-27.468120,153.026820,"[-27.46812, 153.02682]",2


In [124]:
def color(cluster): 
    if cluster == 0: 
        col = 'black'
    elif cluster == 1: 
        col = 'red'
    else: 
        col ='purple'
    return col 


map = folium.Map(location=[-27.47130457718122, 153.02508301342277], zoom_start=13.5)
for la, lo, cluster in zip(lat[0:148], long[0:148], pandasDF['prediction']):
    folium.Marker(location = [la,lo], icon = folium.Icon(color=color(cluster), icon = 'bicycle', icon_color='white', prefix = 'fa')).add_to(map)

map

In [125]:
## 9) Exporter la data frame fitted après élimination de la colonne  features, dans le répertoire path-to-output-data

fitted2 = fitted.drop('features')
fitted2.write.format("csv").save(path_to_output_data)

AnalysisException: path file:/C:/Users/Hosti/exported already exists.;